# Notebook para GSC

Lo primero es instalar las dependencias

In [ ]:
pip install pandas searchconsole matplotlib numpy

Importamos las librerias necesarias

In [ ]:
import os
import math
import numpy as np
import pandas as pd
import searchconsole
import matplotlib.pyplot as plt

Nos autenticamos y damos permiso a google para acceder a gsc

In [ ]:
account = searchconsole.authenticate(client_config='client_secrets.json', serialize='credentials.json')

In [ ]:
Configuramos las variables para hacer funcionar la tool

In [ ]:
brand_name = ''
web = ''
gsc_property = account[web]

Obtenemos todas las queries posibles y manipulamos los datos para que queden como nosotros queremos

In [ ]:
queries_df = gsc_property.query.range('today', days=-7).dimension('query').get().to_dataframe()

queries_df['ctr'] = queries_df['ctr'].map(lambda a: round(a * 10, 2))
queries_df['position'] = queries_df['position'].map(lambda a: round(a, 2))
queries_df['brand'] = queries_df.apply(lambda row: 'No Branded' if row['query'].lower().find(brand_name) == -1 else 'Branded', axis=1)

kws_position = []
for position in queries_df['position']:
    if position <= 3:
        kws_position.append('1-3')
    elif position <= 10:
        kws_position.append('3-10')
    elif position <= 50:
        kws_position.append('10-50')
    elif position <= 100:
        kws_position.append('50-100')
    else:
        kws_position.append('+100')
        
queries_df['top'] = kws_position

Obtenemos el porcentaje de kws branded y no branded

In [ ]:
group_by_brand = queries_df.groupby(['brand']).count()
branded = group_by_brand.iloc[0]['query']
no_branded = group_by_brand.iloc[1]['query']
labels = ['Branded', 'No Branded']
porcentaje = [branded, no_branded]

fig, ax = plt.subplots()
ax.pie(porcentaje, radius=3, autopct='%1.1f%%', center=(4, 4), frame=True)
ax.legend(labels)
plt.axis('off')
plt.show()

print(f'Branded: {percentaje_branded}% | No Branded: {percentaje_no_branded}%')

Obtenemos el CTR dependiendo si las kws son branded o no branded

In [ ]:
brand_queries = queries_df.groupby(['brand']).sum()
branded_queries_ctr = brand_queries.iloc[0]['ctr']
no_branded_queries_ctr = brand_queries.iloc[1]['ctr']

avg_ctr_no_branded = round(no_branded_queries_ctr / group_by_brand.iloc[1]['query'], 2)
avg_ctr_branded = round(branded_queries_ctr / group_by_brand.iloc[0]['query'], 2)

labels = ['Average CTR']

x = np.arange(len(labels))
width = 0.3

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, avg_ctr_branded, width, label='Branded')
rects2 = ax.bar(x + width/2, avg_ctr_no_branded, width, label='No Branded')

ax.set_ylabel('CTR')
ax.set_xticks(x, labels)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

ax.bar_label(rects1, padding=3, labels=[f'{avg_ctr_branded}%'], fontsize=14, label_type='center')
ax.bar_label(rects2, padding=3, labels=[f'{avg_ctr_no_branded}%'], fontsize=14, label_type='center')

fig.tight_layout()
plt.show()

print(f'CTR of Branded Kws: {avg_ctr_branded}% | CTR of No Branded Kws: {avg_ctr_no_branded}%')

In [ ]:
Sacamos un grafico de la distribucion de kws dependiendo si estan en top 1-2, 3-10....

In [ ]:
top_position = queries_df.groupby(['top']).count()

number_kws_1_3 = top_position.iloc[1]['query']
number_kws_3_10 = top_position.iloc[3]['query']
number_kws_10_50 = top_position.iloc[2]['query']
number_kws_50_100 = top_position.iloc[4]['query']
number_kws__100 = top_position.iloc[0]['query']

kws_positions = [number_kws_1_3, number_kws_3_10, number_kws_10_50, number_kws_50_100, number_kws__100]

fig, ax = plt.subplots()
plt.margins(0.1)
x_pos = np.arange(len(kws_positions))
                                       
bar = ax.bar(x_pos, kws_positions, align='center')
ax.bar_label(bar, labels=kws_positions, padding = 3) 
ax.set_xticks(x_pos, labels=['1-3', '3-10', '10-50', '50-100', '+100'])
ax.set_ylabel('Numero de Kws')
ax.set_xlabel('Posiciones')
ax.set_title('Kws Distribution')
plt.axis('off')
plt.show()

In [ ]:
Sacamos el porcentaje de impresiones por dispositivos, Desktop, Mobile o Tablet

In [ ]:
device_df = gsc_property.query.range('today', days=-7).dimension('device').get().to_dataframe()
device_impressions = device_df['impressions']
labels = ['Desktop', 'Mobile', 'Tablet']
fig, ax = plt.subplots()
plt.margins(0.1)
plt.axis('off')
ax.pie(device_impressions, radius=3, labels = labels, center=(4, 4), frame=True, autopct='%1.1f%%')
ax.set_title('Impresiones por dispositivo')
plt.show()